# Medical Chatbot Application

## Introduction

This project is a Flask-based medical chatbot application that uses Natural Language Processing (NLP) models built with Keras. It predicts user intents based on trained models and provides appropriate responses from a predefined dataset stored in a JSON file.
The chatbot interface allows users to interact via a web application, sending messages and receiving context-specific replies. It supports essential features such as login, chatbot responses, and appointment management.

Here is the entire code to look at - 

In [ ]:
import nltk
nltk.download('popular')
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
import random
import json
from keras.models import load_model
from flask import Flask, render_template, request, redirect, url_for

# Load the trained model and supporting files
model = load_model('keras_intent_model.h5')
intents = json.loads(open('data.json').read())
words = pickle.load(open('texts.pkl','rb'))
classes = pickle.load(open('labels.pkl','rb'))

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

def clean_up_sentence(sentence):
    # Tokenize the sentence and lemmatize each word
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    # Convert sentence to bag of words
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print(f"Found in bag: {w}")
    return np.array(bag)

def predict_class(sentence, model):
    # Predict the class of the sentence
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    # Get the response for the predicted intent
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    if len(ints) == 0:  # Check if no intents are predicted
        return "Sorry, I didn't quite understand that. Could you try again?"
    res = get_response(ints, intents)  # Ensure correct function name is used
    return res

# Flask application setup
app = Flask(__name__)
app.static_folder = 'static'

# Home Route (Chatbot interface)
@app.route("/")
def home():
    return render_template("index.html")

# Chatbot Response Route
@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    return chatbot_response(userText)

# Login Route
@app.route("/login", methods=["GET", "POST"])
def login():
    return render_template("login.html")

# Appointments Route (After login)
@app.route("/appointments")
def appointments():
    # This is where you would show or create appointments
    # Placeholder for now
    return render_template("appointments.html")

# Logout Route (Optional)
@app.route("/logout")
def logout():
    return redirect(url_for('home'))

if __name__ == "__main__":
    app.run(debug=True)


Now, let's break down the code to understand it in an easy way.

## Project Setup & Requirements

To run this project, you need the following dependencies:

In [ ]:
# Install required libraries
!pip install flask keras nltk


## 1. Importing Required Libraries
The first step is importing essential libraries for NLP, model handling, and Flask server setup.



In [ ]:
import nltk
nltk.download('popular')
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
import random
import json
from keras.models import load_model
from flask import Flask, render_template, request, redirect, url_for

Explanation:

1. nltk: Used for NLP tasks like tokenization and lemmatization.
2. pickle: Loads preprocessed data such as words and class labels.
3. numpy: Handles numerical computations.
4. random: Generates random responses from predefined responses.
5. json: Parses the intents file containing chatbot data.
6. Flask: Manages the web application and routing.
7. keras.models.load_model: Loads the pre-trained intent classification model.

## 2. Model Loading and Preprocessing

The following code loads the trained model, intents file, and supporting files required for chatbot prediction.

In [ ]:
# Load the trained model and supporting files
model = load_model('keras_intent_model.h5')
intents = json.loads(open('data.json').read())
words = pickle.load(open('texts.pkl','rb'))
classes = pickle.load(open('labels.pkl','rb'))


Explanation:
- model = load_model('keras_intent_model.h5'): Loads the trained Keras model used for intent classification.
- intents = json.loads(open('data.json').read()): Reads the intents file containing response data in JSON format.
- words = pickle.load(open('texts.pkl','rb')): Loads tokenized words used during model training.
- classes = pickle.load(open('labels.pkl','rb')): Loads class labels corresponding to intents.

## Text Preprocessing Functions
The following functions handle text preprocessing tasks, including tokenization, lemmatization, and creating a bag of words representation.

### a. Cleaning Up Sentences

In [ ]:
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

def clean_up_sentence(sentence):
    # Tokenize the sentence and lemmatize each word
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


Explanation:
- WordNetLemmatizer(): Reduces words to their base forms (e.g., "running" → "run").
- nltk.word_tokenize(sentence): Splits the sentence into individual words (tokens).
- lemmatizer.lemmatize(word.lower()): Converts each word to lowercase and lemmatizes it for consistent processing.

### b. Creating a Bag of Words (BoW)

In [ ]:
def bow(sentence, words, show_details=True):
    # Convert sentence to bag of words
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)  
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print(f"Found in bag: {w}")
    return np.array(bag)


Explanation:
1. sentence_words = clean_up_sentence(sentence): Preprocesses the input sentence.
2. bag = [0] * len(words): Initializes a zero-filled list of the same length as the vocabulary.
3. if w == s: bag[i] = 1: Sets the corresponding index to 1 if the word exists in the sentence.
4. np.array(bag): Converts the list into a NumPy array for model compatibility.


## 4. Prediction and Response Functions
These functions handle predicting the intent of the user's input and retrieving the appropriate response.

### a. Predicting the Class of the Sentence

In [ ]:
def predict_class(sentence, model):
    # Predict the class of the sentence
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


Explanation:
- bow(sentence, words, show_details=False): Converts the sentence into a bag of words representation.
- model.predict(np.array([p]))[0]: Predicts the intent using the pre-trained model.
- ERROR_THRESHOLD = 0.25: Sets the minimum confidence threshold for predictions.
- [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]: Filters results to include only those with confidence above the threshold.
- results.sort(key=lambda x: x[1], reverse=True): Sorts the results by probability in descending order.
- return_list.append({"intent": classes[r[0]], "probability": str(r[1])}): Prepares the list of predicted intents with probabilities.

### b. Getting the Response for the Predicted Intent

In [ ]:
def get_response(ints, intents_json):
    # Get the response for the predicted intent
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result


Explanation:
- tag = ints[0]['intent']: Retrieves the predicted intent from the list of intents.
- list_of_intents = intents_json['intents']: Accesses the list of all intents in the intents JSON file.
- random.choice(i['responses']): Chooses a random response from the list of responses for the predicted intent.

### c. Generating Chatbot Response

In [ ]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    if len(ints) == 0:  # Check if no intents are predicted
        return "Sorry, I didn't quite understand that. Could you try again?"
    res = get_response(ints, intents)  # Ensure correct function name is used
    return res


Explanation:
- predict_class(msg, model): Predicts the intent of the user's message.
- if len(ints) == 0:: Checks if no intents were predicted (i.e., the model couldn't recognize the message).
- get_response(ints, intents): Retrieves the corresponding response based on the predicted intent.

## 5. Flask Routes and Web Interface
The following section sets up the routes and user interface using Flask to handle user interactions with the chatbot.

### a. Setting Up the Flask Application



In [ ]:
# Flask application setup
app = Flask(__name__)
app.static_folder = 'static'


Explanation:
- Flask(__name__): Initializes the Flask application.
- app.static_folder = 'static': Specifies the folder for static files like images, CSS, and JavaScript.

### b. Home Route (Chatbot Interface)

In [ ]:
@app.route("/")
def home():
    return render_template("index.html")


Explanation:
- @app.route("/"): Defines the home route for the chatbot interface.
- render_template("index.html"): Renders the index.html template as the landing page for the user interface.

### c. Chatbot Response Route

In [ ]:
@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    return chatbot_response(userText)


Explanation:
- @app.route("/get"): Defines the route for receiving messages from the user.
- userText = request.args.get('msg'): Retrieves the user's input message sent as a query parameter.
- chatbot_response(userText): Passes the message to the chatbot_response function and returns the response.

### d. Login and Appointments Routes

In [ ]:
@app.route("/login", methods=["GET", "POST"])
def login():
    return render_template("login.html")

@app.route("/appointments")
def appointments():
    # This is where you would show or create appointments
    # Placeholder for now
    return render_template("appointments.html")


Explanation:
- @app.route("/login"): Defines the login route for users.
- @app.route("/appointments"): Defines the appointments route for logged-in users. The function currently renders a placeholder page.

### e. Logout Route

In [ ]:
@app.route("/logout")
def logout():
    return redirect(url_for('home'))


Explanation:
- @app.route("/logout"): Defines the logout route for users.
- redirect(url_for('home')): Redirects the user back to the home page after logging out.

### Running the Flask Application

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)


Explanation:
- if __name__ == "__main__":: Ensures the Flask application runs only when the script is executed directly (not when imported as a module).
- app.run(debug=True): Starts the Flask development server in debug mode for easy debugging and live reloading.

## Conclusion
The Medical Chatbot Application is a Flask-based web application designed to assist users by predicting their intents through natural language processing. It utilizes a pre-trained Keras model to identify the intent of user input and generates relevant responses from a predefined set. Here's a brief summary of how the system works:

1. User Interaction: Users send messages through the chatbot interface hosted on the Flask web application.
2. Text Processing: The user's message is preprocessed by tokenizing, lemmatizing, and converting it into a bag of words representation.
3. Intent Prediction: The processed input is passed to the trained model, which predicts the intent of the message based on predefined classes.
4. Response Generation: The system matches the predicted intent with a response from the corresponding intent data in the JSON file.
5. Web Interface: Flask routes manage the user login, chatbot interface, and appointments, ensuring seamless interaction between the user and the system.
6. Web Application: The Flask application renders templates for user interactions, allowing users to chat with the bot, log in, and view appointments.

This structure allows easy integration of NLP models with a web application, providing a powerful tool for user interaction in various domains like healthcare.



## Important Notes

#### Tokenization

What is Tokenization?

Tokenization is the process of breaking down a sentence into smaller units, called tokens. Tokens can be words, subwords, or even characters, but in most cases, tokenization refers to splitting text into individual words or phrases.

Why is Tokenization Important in NLP?
1. Structure: It helps structure raw text into manageable pieces.
2. Analysis: Enables the model to understand the distinct elements (words or symbols) in a sentence.
3. Consistency: Allows for the consistent processing of text by splitting it into parts that can be individually analyzed.

Example:

Let’s take the sentence:

"I am feeling great today!"

After tokenization, it would be split into the following tokens:
["I", "am", "feeling", "great", "today", "!"]

In the Code:

In your code, nltk.word_tokenize(sentence) is used to perform tokenization. It converts the sentence into a list of words that can be further processed.

#### Lemmatization
What is Lemmatization?

Lemmatization is the process of reducing words to their base or root form, known as a lemma. Unlike stemming, which simply removes suffixes to form root words, lemmatization ensures the output is a valid word in the dictionary.

Why is Lemmatization Important in NLP?
- Standardization: Reduces inflected words to a common base form, improving the model's ability to recognize different forms of the same word.
- Meaning Preservation: Lemmatization keeps the meaning of words intact. It ensures that variations of a word, such as "running," "ran," and "runs," are all understood as the base form "run."
- Efficiency: Helps in reducing the number of unique words that need to be handled by the model.

Example:

For the sentence:

"The cats are running and the dogs ran."

After lemmatization, it might convert to:
["The", "cat", "are", "run", "and", "the", "dog", "run"]

In the Code:

In your code, WordNetLemmatizer() is used to lemmatize words. It ensures that variations of words are reduced to their base form before being processed further.

#### Key Intuitions in my Chatbot Code
1. Tokenization: By breaking down the user's sentence into individual tokens, you can more effectively analyze the sentence. The model can look at each token (word) separately to understand its meaning and structure.
2. Lemmatization: Once the sentence is tokenized, lemmatization ensures that variations of the same word (like "running" and "run") are treated as the same, improving the chatbot’s ability to identify intents accurately.

These two techniques are fundamental to making the chatbot more intelligent by ensuring that it understands the core meaning of a sentence without being thrown off by different forms of words or sentence structures.